# Stokes Profile Baseline Training

This notebook trains a baseline model for Stokes profile regression.

The model takes synthesized Stokes profiles (I, Q, U, V) from Surya magnetograms as input
and predicts Stokes profiles that match those synthesized from observed HMI magnetograms.

For this baseline, we train on all pixels' profiles from a single image.

## Set your cuda visible device

**IMPORTANT:** Since we are sharing resources, please make sure that the cuda visible device you put here is the one assigned to your team and your machine.

Here we initialize variables related to Weights and Biases, our online logging system to ensure they are user specific

In [1]:
import os
# Make sure wandb logs are stored in a user-specific directory
# Set writable directories
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["WANDB_DIR"] = "./wandb/wandb_logs"
os.environ["WANDB_CACHE_DIR"] = "./wandb/wandb_cache"
os.environ["WANDB_CONFIG_DIR"] = "./wandb/wandb_config"
# Optional:
os.environ["TMPDIR"] = "./wandb/wandb_tmp"

# Ensure directories exist (optional, wandb usually creates them)
os.makedirs(os.environ["WANDB_DIR"], exist_ok=True)
os.makedirs(os.environ["WANDB_CACHE_DIR"], exist_ok=True)
os.makedirs(os.environ["WANDB_CONFIG_DIR"], exist_ok=True)
os.makedirs(os.environ["TMPDIR"], exist_ok=True)

In [2]:
import sys
from torch.utils.data import DataLoader

import torch
import yaml
import numpy as np

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger, WandbLogger

# Append base path.  May need to be modified if the folder structure changes.
# It gives the notebook access to the wokshop_infrastructure folder.
sys.path.append("../../")
 
# Append Surya path. May need to be modified if the folder structure changes.
# It gives the notebook access to surya's release code.
sys.path.append("../../Surya")

from surya.utils.data import build_scalers  # Data scaling utilities for Surya stacks

torch.set_float32_matmul_precision('medium')

## Download scalers
Surya input data needs to be scaled properly for the model to work and this cell downloads the scaling information.

- If the cell below fails, try running the provided shell script directly in the terminal.
- Sometimes the download may fail due to network or server issues—if that happens, simply re-run the script a few times until it completes successfully.

## Verify GPU setup and clear memory

Before loading data, let's verify the GPU is available and clear any existing memory.

In [3]:
# Verify GPU setup
import torch
if torch.cuda.is_available():
    print(f"✅ CUDA available")
    print(f"   Device count: {torch.cuda.device_count()}")
    print(f"   Current device: {torch.cuda.current_device()}")
    print(f"   Device name: {torch.cuda.get_device_name(0)}")
    print(f"   Memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"   Memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
    # Clear cache to free up memory
    torch.cuda.empty_cache()
    print(f"   ✅ GPU cache cleared")
else:
    print("⚠️  CUDA not available, will use CPU (will be slow)")

✅ CUDA available
   Device count: 1
   Current device: 0
   Device name: NVIDIA A100-SXM4-40GB
   Memory allocated: 0.00 GB
   Memory reserved: 0.00 GB
   ✅ GPU cache cleared


In [4]:
!sh download_scalers.sh

==> Checking assets directory at: /home/qinli/surya_workshop/downstream_apps/template_Qin/assets
==> Downloading scalers.yaml from nasa-ibm-ai4science/core-sdo
Fetching 1 files: 100%|██████████████████████████| 1/1 [00:00<00:00, 893.55it/s]
/home/qinli/surya_workshop/downstream_apps/template_Qin/assets
✓ Done. File is located at: /home/qinli/surya_workshop/downstream_apps/template_Qin/assets/scalers.yaml


## Load configuration

Surya was designed to read a configuration file that defines many aspects of the model
including the data it uses we use this config file to set default values that do not
need to be modified, but also to define values specific to our downstream application

In [5]:
# Configuration paths - modify these if your files are in different locations
config_path = "./configs/config.yaml"

# Load configuration
print("📋 Loading configuration...")
try:
    config = yaml.safe_load(open(config_path, "r"))
    config["data"]["scalers"] = yaml.safe_load(open(config["data"]["scalers_path"], "r"))
    print("✅ Configuration loaded successfully!")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Make sure config.yaml exists in your current directory")
    raise

scalers = build_scalers(info=config["data"]["scalers"])

📋 Loading configuration...
✅ Configuration loaded successfully!


## Define Stokes Profile datasets

This dataset loads Surya-generated magnetograms and observed HMI magnetograms,
then synthesizes Stokes profiles (I, Q, U, V) for all pixels using the
Milne-Eddington forward model.

**Important: We set max_number_of_samples=1 to train on a single image (all pixels' profiles).**

In [6]:
from datasets.stokes_profile_dataset import StokesProfileDataset

# Path to HMI B FITS files directory
hmi_b_dir = "./datasets/hmi.B"

# HMI wavelength range (typically ±0.5 Å around 6173.15 Å)
wavelengths = np.linspace(6172.65, 6173.65, 50)

train_dataset = StokesProfileDataset(
    #### All these lines are required by the parent HelioNetCDFDataset class
    index_path=config["data"]["train_data_path"],
    hmi_b_dir=hmi_b_dir,
    time_delta_input_minutes=config["data"]["time_delta_input_minutes"],
    time_delta_target_minutes=config["data"]["time_delta_target_minutes"],
    n_input_timestamps=config["model"]["time_embedding"]["time_dim"],
    rollout_steps=config["rollout_steps"],
    channels=config["data"]["channels"],
    drop_hmi_probability=config["drop_hmi_probability"],
    use_latitude_in_learned_flow=config["use_latitude_in_learned_flow"],
    scalers=scalers,
    phase="train",
    s3_use_simplecache=True,
    s3_cache_dir="/tmp/helio_s3_cache",
    #### Put your downstream (DS) specific parameters below this line
    wavelengths=wavelengths,
    pixel_batch_size=10000,  # Process pixels in batches to manage memory
    device='cpu',  # Use CPU for synthesis to save GPU memory (data moved to GPU during training)
    max_number_of_samples=1,  # Train on one image (all pixels' profiles)
)

# The Validation dataset changes the index we read
val_dataset = StokesProfileDataset(
    #### All these lines are required by the parent HelioNetCDFDataset class
    index_path=config["data"]["valid_data_path"],  #<---------------- different index path
    hmi_b_dir=hmi_b_dir,
    time_delta_input_minutes=config["data"]["time_delta_input_minutes"],
    time_delta_target_minutes=config["data"]["time_delta_target_minutes"],
    n_input_timestamps=config["model"]["time_embedding"]["time_dim"],
    rollout_steps=config["rollout_steps"],
    channels=config["data"]["channels"],
    drop_hmi_probability=config["drop_hmi_probability"],
    use_latitude_in_learned_flow=config["use_latitude_in_learned_flow"],
    scalers=scalers,
    phase="val",
    s3_use_simplecache=True,
    s3_cache_dir="/tmp/helio_s3_cache",
    #### Put your downstream (DS) specific parameters below this line
    wavelengths=wavelengths,
    pixel_batch_size=10000,
    device='cpu',  # Use CPU for synthesis to save GPU memory (data moved to GPU during training)
    max_number_of_samples=1,  # Validate on one image
)

We also initialize separate training and validation dataloaders. Since we are working in a shared environment, using multiprocessing_context="spawn" helps avoid lockups.

**Note:** For Stokes profiles, we use batch_size=1 since each sample contains all pixels from an image.

In [7]:
batch_size = 1  # Each sample contains all pixels from one image

train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,  # Set to 0 to avoid multiprocessing issues with Stokes synthesis
    pin_memory=False,  # Set to False since data is kept on CPU to save GPU memory
)

val_data_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=0,
    pin_memory=False,  # Set to False since data is kept on CPU to save GPU memory
)

In [8]:
from models.stokes_baseline import StokesBaselineModel

# Initialize model
# Input: [B, 4, n_wavelengths, H, W] - Stokes profiles from Surya
# Output: [B, 4, n_wavelengths, H, W] - Predicted Stokes profiles
n_wavelengths = len(wavelengths)
model = StokesBaselineModel(
    n_wavelengths=n_wavelengths,
    hidden_dim=128,
    use_conv=True,  # Use convolutional layers (set to False for fully-connected)
)

In [9]:
batch = next(iter(train_data_loader))
print(f"Input shape: {batch['stokes_input'].shape}")
print(f"Target shape: {batch['forecast'].shape}")

output = model.forward(batch)
print(f"Output shape: {output.shape}")
print("✅ Model forward pass successful!")

    Batch 1/1: Completed (36 patches)s (100.0%)
    Batch 1/1: Completed (36 patches)s (100.0%)
Input shape: torch.Size([1, 4, 50, 512, 512])
Target shape: torch.Size([1, 4, 50, 512, 512])
Output shape: torch.Size([1, 4, 50, 512, 512])
✅ Model forward pass successful!


## Define your metrics

Metrics are a very important part of training AI models. They provide your models with the quantification of error, which in turn shifts the weights towards better performing models. They also provide a way for you to monitor performance, identify overfitting, and quantify value added.

We now initialize the metrics class which allows you to control what metrics do you want to use as "loss" (i.e. the metrics that backpropagate through your model) and which ones for monitoring performance.

In [10]:
from metrics.stokes_metrics import StokesMetrics

train_loss_metrics = StokesMetrics("train_loss")
train_evaluation_metrics = StokesMetrics("train_metrics")
validation_evaluation_metrics = StokesMetrics("val_metrics")

In [11]:
train_loss_metrics(output, batch["forecast"])

({'mse': tensor(0.4528, grad_fn=<WhereBackward0>)}, [1.0])

In [12]:
train_evaluation_metrics(output, batch["forecast"])

({'mae': tensor(0.3476),
  'mse_I': tensor(1.8100),
  'mse_Q': tensor(5.3749e-05),
  'mse_U': tensor(0.0008),
  'mse_V': tensor(0.0003)},
 [1.0, 1.0, 1.0, 1.0, 1.0])

In [13]:
validation_evaluation_metrics(output, batch["forecast"])

({'mse': tensor(0.4528),
  'mae': tensor(0.3476),
  'mse_I': tensor(1.8100),
  'mse_Q': tensor(5.3749e-05),
  'mse_U': tensor(0.0008),
  'mse_V': tensor(0.0003),
  'mae_I': tensor(1.3423),
  'mae_Q': tensor(0.0054),
  'mae_U': tensor(0.0254),
  'mae_V': tensor(0.0173)},
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [14]:
from lightning_modules.pl_stokes_baseline import StokesLightningModule

In [15]:
L.seed_everything(42, workers=True)

Seed set to 42


42

In [16]:
metrics = {
    'train_loss': train_loss_metrics,
    'train_metrics': train_evaluation_metrics,
    'val_metrics': validation_evaluation_metrics
}

learning_rate = 1e-3
lit_model = StokesLightningModule(model, metrics, lr=learning_rate, batch_size=batch_size)

In [17]:
project_name = "stokes_profile_regression"
run_name = "baseline_experiment_QL"

wandb_logger = WandbLogger(
    entity="surya_handson",
    project=project_name,
    name=run_name,
    log_model=False,
    save_dir="./wandb/wandb_tmp",
)

csv_logger = CSVLogger("runs", name="stokes_baseline")

## Initialize trainer

With the loggers done, now the trainer needs to be defined. The trainer defines several properties of your training run. Here we define:

- The max number of epochs (one epoch represents your model seeing your entire training dataset).
- Define where the training run will take place (auto uses the GPU if possible, if not, CPU).
- The loggers.
- The callbacks (here we save the model with the lowest validation loss).
- Logging frequency (because we are working with a small dataset it needs to be small).

In [18]:
max_epochs = 100

# -------------------------------------------------------------------------
# Trainer
# -------------------------------------------------------------------------
trainer = L.Trainer(
    max_epochs=max_epochs,
    accelerator="auto",
    devices="auto",
    logger=[wandb_logger, csv_logger],
    callbacks=[
        ModelCheckpoint(
            monitor="val_loss",
            mode="min",
            save_top_k=1,
        )
    ],
    log_every_n_steps=2,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


## Fit the model

Finally we fit the model. We pass the Lightning module, and our dataloaders.

In [19]:
trainer.fit(lit_model, train_data_loader, val_data_loader)

wandb: WARNING The anonymous setting has no effect and will be removed in a future version.
wandb: Currently logged in as: ql47 (surya_handson) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name  | Type                | Params | Mode  | FLOPs
--------------------------------------------------------------
0 | model | StokesBaselineModel | 52.5 K | train | 0    
--------------------------------------------------------------
52.5 K    Trainable params
0         Non-trainable params
52.5 K    Total params
0.210     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/envs/surya_ws/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Loaded disambiguation file for timestamp 20110116_000000
Applied disambiguation: 5838337 pixels flipped (out of 16777216 total)
    Batch 1/1: Completed (36 patches)s (100.0%)
    Batch 1/1: Completed (36 patches)s (100.0%)


...

Validation: |          | 0/? [00:00<?, ?it/s]

Loaded disambiguation file for timestamp 20110116_000000
Applied disambiguation: 5838337 pixels flipped (out of 16777216 total)
    Batch 1/1: Completed (36 patches)s (100.0%)
    Batch 1/1: Completed (36 patches)s (100.0%)
    Batch 1/1: Completed (36 patches)s (100.0%)
    Batch 1/1: Completed (36 patches)s (100.0%)


Validation: |          | 0/? [00:00<?, ?it/s]

Loaded disambiguation file for timestamp 20110116_000000
Applied disambiguation: 5838337 pixels flipped (out of 16777216 total)
    Batch 1/1: Completed (36 patches)s (100.0%)
    Batch 1/1: Completed (36 patches)s (100.0%)


`Trainer.fit` stopped: `max_epochs=100` reached.


## Conclusion

With this we have now integrated our dataset, dataloaders, metrics, and baseline into an end-to-end training loop for Stokes profile regression. The model learns to predict Stokes profiles from Surya magnetograms that match those synthesized from observed HMI magnetograms.

The next step could be to:
- Train on more images
- Use a more sophisticated model architecture
- Fine-tune Surya for this task

In [ ]:
# Get the checkpoint callback to access the saved model path
checkpoint_callback = None
for callback in trainer.callbacks:
    if isinstance(callback, ModelCheckpoint):
        checkpoint_callback = callback
        break

if checkpoint_callback and checkpoint_callback.best_model_path:
    best_checkpoint_path = checkpoint_callback.best_model_path
    print(f"✅ Best model checkpoint saved at:")
    print(f"   {best_checkpoint_path}")
    print(f"\n📊 Best validation loss: {checkpoint_callback.best_model_score:.6f}")
else:
    print("⚠️  No checkpoint found. Make sure training completed successfully.")

## Load model for testing/inference

To use the trained model for testing or inference, load it from the checkpoint:

In [ ]:
# Load the trained model from checkpoint for testing
if checkpoint_callback and checkpoint_callback.best_model_path:
    # Load the model using Lightning's load_from_checkpoint
    # This automatically loads the model architecture and weights
    trained_model = StokesLightningModule.load_from_checkpoint(
        checkpoint_callback.best_model_path,
        model=model,  # Pass the model architecture
        metrics=metrics,  # Pass the metrics
        lr=learning_rate,
        batch_size=batch_size
    )
    
    # Set to evaluation mode
    trained_model.eval()
    trained_model.freeze()  # Freeze all parameters
    
    print("✅ Model loaded successfully!")
    print(f"   Model is ready for inference/testing")
    print(f"\n   To use the model for prediction:")
    print(f"   with torch.no_grad():")
    print(f"       predictions = trained_model(batch)")
else:
    print("⚠️  Cannot load model: checkpoint not found")

## Test the loaded model

Test the loaded model on a validation sample to verify it works:

In [ ]:
# Test the loaded model on a validation sample
if checkpoint_callback and checkpoint_callback.best_model_path:
    # Get a validation batch
    val_batch = next(iter(val_data_loader))
    
    # Move batch to device (if using GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trained_model = trained_model.to(device)
    
    # Move input to device
    val_batch['stokes_input'] = val_batch['stokes_input'].to(device)
    
    # Run inference
    with torch.no_grad():
        predictions = trained_model(val_batch)
    
    print(f"✅ Inference successful!")
    print(f"   Input shape: {val_batch['stokes_input'].shape}")
    print(f"   Prediction shape: {predictions.shape}")
    print(f"   Target shape: {val_batch['forecast'].shape}")
    
    # Compute test metrics
    test_metrics = validation_evaluation_metrics(predictions.cpu(), val_batch['forecast'].cpu())
    print(f"\n📊 Test metrics on validation sample:")
    for key, value in test_metrics[0].items():
        print(f"   {key}: {value.item():.6f}")
else:
    print("⚠️  Cannot test model: checkpoint not found")

## Conclusion

With this we have now integrated our dataset, dataloaders, metrics, and baseline into an end-to-end training loop for Stokes profile regression. The model learns to predict Stokes profiles from Surya magnetograms that match those synthesized from observed HMI magnetograms.

**The trained model checkpoint is saved and can be loaded for testing/inference using the code above.**

The next step could be to:
- Train on more images
- Use a more sophisticated model architecture
- Fine-tune Surya for this task
- Use the saved checkpoint for evaluation on test data